# Aufgabe 15

__a)__ Da beim _kNN_-Algorithmus muss die (euklidische) Norm zwischen den Test und Traningsdatenpunkten berechnet werden. Weichen die Größenordnungen der Attribute stark voneinander ab, kann es z.B. 
zu Rundungsfehlern kommen. Zudem ist der Abstandsbegriff sinnvoller, wenn die einzelnen Skalen normiert werden, da dann relative Abstände für alle Attribute gleich gewichtet werden.

__b)__ Der Algorithmus wird als _lazy learner_ bezeichnet, da er eigentlich gar nicht lernt. Der Trainingsdatensatz wird einfach nur abgespeichert und für jeden neuen Testdatensatz die k nächsten Nachbarn neu berechnet werden. Die Laufzeit der Lernphase ist verschwindend klein, während die Anwendungsphase lang dauert. Bei der _SVM_ ist es genau andersherum. 

__c)__ 

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from collections import Counter
import matplotlib.pyplot as plt
from ml import plots
%matplotlib inline
from graphviz import Source
from matplotlib.colors import ListedColormap

Die Klassenstruktur:

In [ ]:
class KNN:
    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        self.training_data = X 
        self.training_labels = y

    def predict(self, X):
        #calculate the eucleadian distance (ignore the root, 
        #cause its a monoton function)
        #between each test event and each training event
        distance = (-2 * np.dot(X, self.training_data.T) 
                    + np.sum(X**2, axis=1)[:, np.newaxis] 
                    + np.sum(self.training_data**2, axis=1)[np.newaxis, :])
        
        #generate matrix with labels of the k nearest neighbours
        labels = self.training_labels.values[(np.argsort(distance))[:, :self.k]]
        
        #most common label of the k nearest neighbours as 
        #prediction for each test event
        prediction = []
        for i in range(np.shape(labels)[0]):
            count = Counter(labels[i, :])
            prediction.append(count.most_common(1)[0][0])

        return prediction

__d)__ Bringe zunächst die Daten in die benötigte Form: 

In [ ]:
#read hdf5 file
neutrino_signal = pd.read_hdf('NeutrinoMC.hdf5', key = 'Signal')

#select the accepted events
neutrino_signal = neutrino_signal[neutrino_signal.AcceptanceMask]

#delete the energy and the acceptancemask (not relevant for this task)
neutrino_signal = neutrino_signal.drop(columns = ['Energy', 'AcceptanceMask'])

#reset the index of the DataFrame
neutrino_signal = neutrino_signal.reset_index(drop = True)

#add label to the signal events
neutrino_signal['label'] = Series(data = ['signal' for i in neutrino_signal.x])

Das gleiche für die Untergrundevents: 

In [ ]:
neutrino_background = pd.read_hdf('NeutrinoMC.hdf5', key = 'Background')
neutrino_background['label'] = Series(data = ['background' for i in 
                                              neutrino_background.x])

Eine Funktion um einen gewünschten gemischten Datensatz aus Signal und Untergrund zu erstellen:

In [ ]:
def mix_sample(signal_events, background_events, n_signal, n_background):
    data_set = pd.concat([background_events.sample(n_background), 
                          signal_events.sample(n_signal)], 
                         ignore_index=True)
    X = data_set.drop(columns = 'label')
    y = data_set['label']
    return X, y

Funktionen für Reinheit usw:

In [ ]:
#Reinheit
def precision(true_pos, false_pos):
    return len(true_pos) / (len(true_pos) + len(false_pos))

#Effizienz
def recall(true_pos, false_neg):
    return len(true_pos) / (len(true_pos) + len(false_neg))

#Signifikanz
def significance(true_pos, false_pos):
    return len(true_pos) / np.sqrt(len(true_pos) + len(false_pos))

Generiere den Trainings- und Testdatensatz:

In [ ]:
X_training, y_training = mix_sample(neutrino_signal, neutrino_background, 5000, 5000)
X_test, y_test = mix_sample(neutrino_signal, neutrino_background, 10000, 20000)

Ab hier ist das Vorgehen für die Aufgabenteile d)-f) analog, wesegen eine Funktion für die Prozedur geschrieben wird:

In [ ]:
def procedure(k, X_training, y_training, X_test, y_test):
    #use the knn algorithm 
    knn = KNN(k = k)
    knn.fit(X = X_training, y = y_training)
    prediction = knn.predict(X = X_test)
    
    #add results to test data set
    X_test['prediction'] = Series(prediction)
    X_test['truth'] = y_test
    
    
    #calculate true positive etc
    true_positive  = X_test[(X_test.truth == 'signal') & 
                            (X_test.prediction == 'signal')]
    
    true_negative  = X_test[(X_test.truth == 'background') & 
                            (X_test.prediction == 'background')]
    
    false_positive = X_test[(X_test.truth == 'background') & 
                            (X_test.prediction == 'signal')]
    
    false_negative = X_test[(X_test.truth == 'signal') & 
                            (X_test.prediction == 'background')]
    
    #calculate precision etc
    precision_knn = precision(true_positive, false_positive)
    recall_knn = recall(true_positive, false_negative)
    significance_knn = significance(true_positive, false_positive)

    print(f'Reinheit: \t{precision_knn}\nEffizienz: \t{recall_knn}\nSignifikanz: \t{significance_knn}')

Hier nun KNN Algorithmus mit $k = 10$:

In [ ]:
procedure(10, X_training, y_training, X_test, y_test)

__e)__ Nun $log_{10}(N)$ statt $N$:

In [ ]:
neutrino_signal_e = neutrino_signal
neutrino_signal_e['log10NumberOfHits'] = np.log10(neutrino_signal['NumberOfHits'])
neutrino_signal_e = neutrino_signal_e.drop(columns = 'NumberOfHits')

neutrino_background_e = neutrino_background
neutrino_background_e['log10NumberOfHits'] = np.log10(neutrino_background['NumberOfHits'])
neutrino_background_e = neutrino_background_e.drop(columns = 'NumberOfHits')

Nun das gleiche wie oben: 

In [ ]:
X_training, y_training = mix_sample(neutrino_signal_e, 
                                    neutrino_background_e, 
                                    5000, 5000)
X_test, y_test = mix_sample(neutrino_signal_e, 
                            neutrino_background_e, 
                            10000, 20000)

In [ ]:
procedure(10, X_training, y_training, X_test, y_test)

Kommentar: Alle Attribute werden besser. Wie oben erwähnt ist dies durch eine Umskalierung des Attributs 'Hits' bedingt. 

__f)__ Nun das ganze mit $k = 20$:

In [ ]:
X_training, y_training = mix_sample(neutrino_signal, neutrino_background, 5000, 5000)
X_test, y_test = mix_sample(neutrino_signal, neutrino_background, 10000, 20000)

In [ ]:
procedure(20, X_training, y_training, X_test, y_test)

Kommentar: Alle Attribute werden schlechter. 

# Aufgabe 16

Der handschriftliche Teil der Aufgabe befindet sich am Ende der pdf. 

In [ ]:
X = DataFrame()
X['temperature'] = Series([29.4, 26.7, 28.3, 21.1, 20, 18.3, 17.8, 22.2, 20.6, 23.9, 23.9, 22.2, 27.2, 21.7])
X['report'] = Series([2, 2, 1, 0, 0, 0, 1, 2, 2, 0, 2, 1, 1, 0])
X['humidity'] = Series([85, 90, 78, 96, 80, 70, 65, 95, 70, 80, 70, 90, 75, 80])
X['wind'] = Series([False, True, False, False, False, True, True, False, False, False, True, True, False, True])
#X

y = DataFrame({'football': Series([False, False, True, True, True, False, True, False, True, True, True, True, True, False])})

In [ ]:
def entropy(y, splits = [[True for y in range(len(y))]]):
    H = []
    for split in splits:
        entropy = 0
        values = Counter(y[split]).most_common()
        for value in values:
            entropy -= value[1] / len(y[split]) * np.log2(value[1] / len(y[split]))
        H.append(entropy)    
    return np.array(H)      

def information_gain(y, splits):
    return entropy(y) - entropy(y, splits)

In [ ]:
print(entropy(y.football))

In [ ]:
report_split = [1, 2, 3]
report_splits = [X.report <= report for report in report_split]

report_information_gain = information_gain(y.football, report_splits)
plt.plot(report_split, report_information_gain, 'ro')
plt.xticks([1, 2, 3])
plt.xlabel('Schnitt Wetterbericht')
plt.ylabel('Informationsgewinn')
None

In [ ]:
humidity_split = np.linspace(min(X.humidity), max(X.humidity), 200)
humidity_splits = [X.humidity <= H for H in humidity_split]

H_information_gain = information_gain(y.football, humidity_splits)
plt.plot(humidity_split, H_information_gain, 'r-')
plt.xlabel('Schnitt-Luftfeuchtigkeit')
plt.ylabel('Informationsgewinn')
None

In [ ]:
temperature_split = np.linspace(min(X.temperature), max(X.temperature), 200)
temperature_splits = [X.temperature <= T for T in temperature_split]


T_information_gain = information_gain(y.football, temperature_splits)
plt.plot(temperature_split, T_information_gain, 'r-')
plt.xlabel('Schnitt-Temperatur')
plt.ylabel('Informationsgewinn')
None

Das Attribut Temperatur eignet sich am besten.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

discrete_cmap = ListedColormap(['xkcd:red', 'xkcd:blue'])
clf = DecisionTreeClassifier(max_depth=10, criterion='entropy')
clf.fit(X, y) 
tree.export_graphviz(clf, out_file = 'tree.dot')

# Aufgabe 17

Wie sollten nicht-numerische Datentypen wie beispielsweise Strings vor der Analyse
behandelt werden müssen?
b) Kann es hilfreich sein Attribute zu normieren? Wenn ja, wieso?
c) Wie kann mit Lücken in den Daten oder NaNs und Infs verfahren werden?
d) Was ist beim Zusammenführen von Datensätzen zu beachten?
e) Welche Attribute sollten vor dem Trainieren des Klassifizierers aus dem Datensatz
entfernt werden. Wie kann dabei eine Reduktion redundanter Informationen er-
reicht werden? Was mum
ss speziell bei simulationsbasierten Methoden berücksichtigt
werden?

__a)__ Aus den Strings sollten vergleichbare Attribute generiert werden, z.B. die Länge oder die Anzahl an bestimmten Buchstaben. Falls die Strings nur eine Eigenschaft angeben, kann diese einer Zahl zugeordnet werden.

__b)__ Dies kann z.B. für den kNN Algorithmus relevant sein, wie oben bereits erklärt. 

__c)__ Man kann diese Attribute oder den jeweiligen Datenpunkt einfach weglassen. Oder interpretieren, was z.B. Infs in dem Kontext bedeuten und dann entsprechend einen anderen Wert zuweisen. 

__d)__ Die Datensätze müssen über die selben Attribute verfügen. 

__e)__ Attribute, die für alle Daten gleich sind enthalten z.B. keinerlei Information. Manche Daten stehen vielleicht in einem funktionellen Zusammenhang zueinander. Einer Reduktion der Attribute kann z.B. mit der PCA erreicht werden. 